In [1]:
import tensorflow as tf

In [2]:
print(tf.__version__)

2.2.0


In [3]:
from keras.layers import Input, Embedding, Flatten, Dot, Dense, Concatenate
from keras.models import Model

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import warnings

warnings.filterwarnings('ignore')
%matplotlib inline

In [5]:
data=pd.read_csv('../petfinder_data/favorites_sample_small.csv', delimiter=';',error_bad_lines=False)

In [6]:
data.head(5)

,user_id,favorited_at,id,organization_id,organization_active,organization_enabled,organization_display_id,import_on,import_software,organization_type,...,good_with_other_animals,good_with_others_notes,house_trained,altered,declawed,special_needs,shots_current,num_photos,num_aifs,num_videos
0,4369629,2020-01-01 00:00:04,46861637,44719,1,1,OH1073,0,NaN,Rescue Group / Foster-Based,...,unknown,NaN,0,1,0,1,1,5,11,0
1,4369629,2020-01-01 00:00:07,46898514,35651,1,1,OH745,0,NaN,Rescue Group / Foster-Based,...,unknown,NaN,0,1,0,0,1,2,0,0
2,4112169,2020-01-01 00:00:08,46980809,40518,1,1,NY1044,0,NaN,Rescue Group / Foster-Based,...,unknown,NaN,1,1,0,0,1,6,0,0
3,4369680,2020-01-01 00:00:12,46956517,29207,1,1,FL806,0,NaN,NaN,...,unknown,NaN,0,0,0,0,1,1,1,0
4,4369681,2020-01-01 00:00:13,46860620,48762,1,1,TN914,1,Petfinder Import Standard Template,Rescue Group / Foster-Based,...,unknown,NaN,0,0,0,0,0,3,26,0
5,4369629,2020-01-01 00:00:16,46898681,49615,1,1,OH1248,0,NaN,Rescue Group / Foster-Based,...,yes,foster has other dogs and cats,0,1,0,0,1,2,5,0
6,4369489,2020-01-01 00:00:17,46559227,26457,1,1,AR96,0,NaN,NaN,...,unknown,NaN,0,1,0,0,1,6,1,0
7,4369629,2020-01-01 00:00:17,46898698,49615,1,1,OH1248,0,NaN,Rescue Group / Foster-Based,...,yes,foster home has other dogs and cats,1,0,0,0,1,1,2,0
8,4369629,2020-01-01 00:00:18,46898687,49615,1,1,OH1248,0,NaN,Rescue Group / Foster-Based,...,yes,foster has other dogs and cats,0,1,0,0,1,1,3,0
9,4369682,2020-01-01 00:00:20,46934463,42387,1,1,NC823,0,NaN,NaN,...,unknown,NaN,0,0,0,0,1,6,2,0


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140969 entries, 0 to 140968
Data columns (total 56 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   user_id                         140969 non-null  int64  
 1   favorited_at                    140969 non-null  object 
 2   id                              140969 non-null  int64  
 3   organization_id                 140969 non-null  int64  
 4   organization_active             140969 non-null  int64  
 5   organization_enabled            140969 non-null  int64  
 6   organization_display_id         140969 non-null  object 
 7   import_on                       140969 non-null  int64  
 8   import_software                 61836 non-null   object 
 9   organization_type               109264 non-null  object 
 10  location_type                   140969 non-null  object 
 11  is_public_location              140969 non-null  int64  
 12  state_code      

In [8]:
data['Description_Flag'] = np.where(data['pet_description'].notnull(), 1, 0)

In [9]:
data['organization_type'].fillna("Unknown", inplace = True) 

In [10]:
data.head(10)

,user_id,favorited_at,id,organization_id,organization_active,organization_enabled,organization_display_id,import_on,import_software,organization_type,...,good_with_others_notes,house_trained,altered,declawed,special_needs,shots_current,num_photos,num_aifs,num_videos,Description_Flag
0,4369629,2020-01-01 00:00:04,46861637,44719,1,1,OH1073,0,NaN,Rescue Group / Foster-Based,...,NaN,0,1,0,1,1,5,11,0,1
1,4369629,2020-01-01 00:00:07,46898514,35651,1,1,OH745,0,NaN,Rescue Group / Foster-Based,...,NaN,0,1,0,0,1,2,0,0,1
2,4112169,2020-01-01 00:00:08,46980809,40518,1,1,NY1044,0,NaN,Rescue Group / Foster-Based,...,NaN,1,1,0,0,1,6,0,0,1
3,4369680,2020-01-01 00:00:12,46956517,29207,1,1,FL806,0,NaN,Unknown,...,NaN,0,0,0,0,1,1,1,0,1
4,4369681,2020-01-01 00:00:13,46860620,48762,1,1,TN914,1,Petfinder Import Standard Template,Rescue Group / Foster-Based,...,NaN,0,0,0,0,0,3,26,0,0
5,4369629,2020-01-01 00:00:16,46898681,49615,1,1,OH1248,0,NaN,Rescue Group / Foster-Based,...,foster has other dogs and cats,0,1,0,0,1,2,5,0,1
6,4369489,2020-01-01 00:00:17,46559227,26457,1,1,AR96,0,NaN,Unknown,...,NaN,0,1,0,0,1,6,1,0,1
7,4369629,2020-01-01 00:00:17,46898698,49615,1,1,OH1248,0,NaN,Rescue Group / Foster-Based,...,foster home has other dogs and cats,1,0,0,0,1,1,2,0,1
8,4369629,2020-01-01 00:00:18,46898687,49615,1,1,OH1248,0,NaN,Rescue Group / Foster-Based,...,foster has other dogs and cats,0,1,0,0,1,1,3,0,1
9,4369682,2020-01-01 00:00:20,46934463,42387,1,1,NC823,0,NaN,Unknown,...,NaN,0,0,0,0,1,6,2,0,1


In [11]:
print("Unique Values in `Flag_Detail` => {}".format(data.Description_Flag.unique()))

Unique Values in `Flag_Detail` => [1 0]


In [12]:
data_des = data[[
    'Description_Flag'
    ,'pet_description']].copy()

data_des.head()

,Description_Flag,pet_description
0,1,Meet Elsa!Elsa came to us as a stray. She had ...
1,1,Coco came into the rescue a couple years ago w...
2,1,AGE: 1 YearSEX: Neutered MaleWEIGHT: 50 Pounds...
3,1,Very sweet puppy will neuter micrchip and give...
4,0,NaN


In [13]:
data2 = data[[
    'user_id'
    ,'id'
    ,'num_aifs'
    , 'Description_Flag'
    , 'mixed_breed']].copy()

In [14]:
datatest = data[[
    'user_id'
    ,'id'
    ,'Description_Flag'
    ,'mixed_breed'
    ,'animal_type'
    ,'age'
    ,'state_code'
    ,'primary_breed'
    ,'organization_type'
    ,'gender'
    ,'size'
    ,'num_aifs'
    ,'house_trained'
    ,'coat_length'
    ,'num_videos'
    ,'num_photos'
    ,'good_with_dogs'
    ,'good_with_cats'
    ,'good_with_children']].copy()

In [15]:
user_enc = LabelEncoder()
# can later use user_enc like so user_enc.inverse_transform([24230]) to get original user_id
data2['user'] = user_enc.fit_transform(data2['user_id'].values)

pet_enc = LabelEncoder()
data2['pet'] = pet_enc.fit_transform(data2['id'].values)

n_users = data2['user'].nunique()
print(n_users)
n_favorites = data2['pet'].nunique()
print(n_favorites)

24231
63307


In [16]:
# validate encodings start from 0 and index to n_users-1 and n_favorites-1
print(data2.user.min())
print(data2.user.max())
print(data2.pet.min())
print(data2.pet.max())

0
24230
0
63306


In [17]:
train, test = train_test_split(data2, test_size=0.2, random_state=42)

In [18]:
print(train)

        user_id        id  num_aifs  Description_Flag  mixed_breed   user  \
78111   4385007  46212165        10                 1            1  18723   
84237   4385930  46975330         0                 0            0  19139   
47676   4377578  46950410         1                 1            1  15342   
1170    4369877  46941845         0                 1            0  11931   
102441  4342757  46979281         1                 1            1  10092   
...         ...       ...       ...               ...          ...    ...   
110268  4391200  46964272         2                 0            0  21498   
119879  4118284  46262196         8                 1            1   4584   
103694  3908620  46568984         4                 0            0   2536   
131932  4215592  47000204         4                 1            1   6521   
121958  4149300  46809506         0                 1            1   5122   

          pet  
78111   14284  
84237   51634  
47676   45693  
1170    444

In [19]:
fav_input = Input(shape=[1], name="Fav-Input")
fav_embedding = Embedding(n_favorites, 50, name="Fav-Embedding")(fav_input)
fav_vec = Flatten(name="Flatten-Books")(fav_embedding)


user_input = Input(shape=[1], name="User-Input")
user_embedding = Embedding(n_users, 50, name="User-Embedding")(user_input)
user_vec = Flatten(name="Flatten-Users")(user_embedding)


prod = Dot(name="Dot-Product", axes=1)([fav_vec, user_vec])
model = Model([user_input, fav_input], prod)
model.compile('adam', 'mean_squared_error')

In [20]:
#Model error
#https://github.com/tensorflow/tensorflow/issues/23698

from keras.models import load_model

if os.path.exists('model.h5'):
    model = load_model('model.h5')
else:
    history = model.fit([train.user, train.pet], train.num_aifs, epochs=20, verbose=1)
    model.save('model.h5')
    plt.plot(history.history['loss'])
    plt.xlabel("Epochs")
    plt.ylabel("Training Error")

In [21]:
# making a recommendation
select_user = 51
pet_data = np.array(list(set(train.pet)))
user = np.array([select_user for i in range(len(pet_data))])
predictions = model.predict([user, pet_data])
predictions = np.array([a[0] for a in predictions])
recommended_pet_ids = (-predictions).argsort()[:15]

# use pet_enc to get the original pet's id as it exists in the petfinder dataset
original_ids = []
for pet_id in recommended_pet_ids:
    original_ids.append(pet_enc.inverse_transform([pet_data[pet_id]])[0])
    
print([pet_data[x] for x in recommended_pet_ids])
print("----------------------------------------------------------------------")
print(original_ids)
print("----------------------------------------------------------------------")
print(predictions[recommended_pet_ids])

[26295, 49264, 42457, 10818, 28759, 19761, 6420, 11210, 17147, 28394, 1600, 31849, 37775, 57249, 14115]
----------------------------------------------------------------------
[46696243, 46967218, 46926123, 45923485, 46749546, 46492776, 45340147, 45958357, 46376493, 46742072, 44307546, 46803802, 46880509, 47006052, 46203392]
----------------------------------------------------------------------
[14.049809  13.4470215 11.652007   9.654191   9.171172   9.023783
  8.644172   8.633325   8.501139   8.099102   8.091631   7.997397
  7.779311   7.725931   7.6163874]


In [22]:
# view all the pets this user liked
datatest[datatest.user_id == user_enc.inverse_transform([select_user])[0]]

,user_id,id,Description_Flag,mixed_breed,animal_type,age,state_code,primary_breed,organization_type,gender,size,num_aifs,house_trained,coat_length,num_videos,num_photos,good_with_dogs,good_with_cats,good_with_children
27232,381174,46999325,1,1,Dog,Young,PA,German Shepherd Dog,Private Animal Shelter,Female,Large,3,1,Medium,0,3,unknown,unknown,unknown
67487,381174,47001682,0,0,Dog,Adult,PA,German Shepherd Dog,Rescue Group / Foster-Based,Female,Medium,0,0,NaN,0,2,unknown,unknown,unknown
68972,381174,47002641,1,1,Dog,Adult,PA,Beagle,Private Animal Shelter,Male,Medium,0,0,Short,0,3,yes,unknown,unknown
69010,381174,47002552,1,1,Dog,Young,PA,Shepherd,Private Animal Shelter,Male,Medium,3,0,Short,0,2,yes,unknown,unknown
69744,381174,46189556,1,1,Dog,Adult,PA,Terrier,Private Animal Shelter,Female,Medium,2,0,Short,0,2,unknown,no,unknown
69800,381174,47002546,1,1,Dog,Young,PA,Shepherd,Private Animal Shelter,Male,Medium,0,0,Short,0,4,yes,unknown,unknown
70187,381174,46999411,1,1,Dog,Young,PA,Retriever,Rescue Group / Foster-Based,Male,Large,2,1,Medium,0,5,yes,yes,yes
70305,381174,46808976,1,1,Dog,Baby,PA,Belgian Shepherd / Malinois,Rescue Group / Foster-Based,Male,Medium,1,1,Short,0,3,yes,yes,unknown
104388,381174,47015572,1,0,Dog,Adult,PA,Greater Swiss Mountain Dog,Rescue Group / Foster-Based,Female,Large,0,1,NaN,0,3,yes,unknown,yes


In [23]:
# get list of pets the user has already liked
already_liked = datatest[datatest.user_id == user_enc.inverse_transform([select_user])[0]].id.tolist()
filtered_recs = [x for x in original_ids if x not in already_liked]
print ("recommendations after removing pet's user already liked ...")
print (filtered_recs)

recommendations after removing pet's user already liked ...
[46696243, 46967218, 46926123, 45923485, 46749546, 46492776, 45340147, 45958357, 46376493, 46742072, 44307546, 46803802, 46880509, 47006052, 46203392]


In [24]:
# view the user's recommended pets
datatest[datatest.id.isin(filtered_recs)].drop_duplicates(subset="id", keep="first")

,user_id,id,Description_Flag,mixed_breed,animal_type,age,state_code,primary_breed,organization_type,gender,size,num_aifs,house_trained,coat_length,num_videos,num_photos,good_with_dogs,good_with_cats,good_with_children
325,4345903,45923485,1,1,Dog,Baby,MD,Feist,Unknown,Male,Small,128,0,NaN,0,1,yes,unknown,unknown
2146,4362774,46967218,1,1,Dog,Young,KY,Maltese,Rescue Group / Foster-Based,Female,Small,139,1,Medium,0,3,yes,unknown,no
2420,4370109,46926123,1,0,Dog,Young,PA,Newfoundland Dog,Rescue Group / Foster-Based,Female,Large,119,1,Long,0,2,yes,unknown,yes
2780,4183980,46742072,1,1,Dog,Adult,CT,English Bulldog,Rescue Group / Foster-Based,Male,Medium,43,1,Short,0,3,yes,yes,yes
5309,4183980,45340147,1,1,Cat,Baby,CT,Siamese,Rescue Group / Foster-Based,Female,Medium,491,1,Short,0,3,yes,yes,yes
5823,4370763,44307546,1,1,Dog,Baby,FL,Basenji,Unknown,Male,Small,108,1,NaN,0,1,yes,unknown,yes
8741,4371330,46492776,1,1,Dog,Adult,IL,Bichon Frise,Rescue Group / Foster-Based,Male,Small,52,0,NaN,0,2,unknown,no,unknown
9832,3389158,46803802,1,1,Dog,Young,CA,Dachshund,Unknown,Female,Small,600,1,Medium,0,1,yes,yes,yes
12870,4371973,45958357,1,1,Dog,Young,MO,Collie,Rescue Group / Foster-Based,Female,Medium,74,1,NaN,0,2,yes,unknown,yes
13811,2047606,46880509,1,0,Dog,Baby,CA,Beagle,Rescue Group / Foster-Based,Female,Small,48,0,Short,0,3,yes,unknown,unknown


In [25]:
if os.path.exists('user_mappings.json') == False and os.path.exists('pet_mappings.json') == False:
    print ("creating mapping files ...")
    # create dicts which hold the mappings from encoded id's -> original id's in petfinder dataset
    user_dict = {}
    pet_dict = {}

    for user_id in data2.user.unique():
        # original id --> encoded id
        user_dict[int(user_enc.inverse_transform([user_id])[0])] = int(user_id)
    
    for pet_id in data2.pet.unique():
         # encoded id --> original id
        pet_dict[int(pet_id)] = int(pet_enc.inverse_transform([pet_id])[0])

    # create json "mapping" files
    import json
    with open('user_mappings.json', 'w') as fp:
        json.dump(user_dict, fp)

    with open('pet_mappings.json', 'w') as fp:
        json.dump(pet_dict, fp)
else:
    print("mapping files already created!")

mapping files already created!
